**Analysizing Etobiocke**

In [10]:
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

**Getting Data from Foursquare**

In [11]:
df1 = df[df['Borough'] == 'Etobicoke']
version = 20180605

locator = Nominatim()
location = locator.geocode('Etobicoke,Toronto,Ontario')
lat = location.latitude
long = location.longitude
clientid = 'D5U52LKULW0C51MXLUBQJFEQYWCVIXGXFHY24CTFMHIUKKJY'
clientsecret = 'KLQOMKFV1LZGRJGC4TMNU10PNHBJWN51GNSDMLOEKHFGZSLN'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    clientid,clientsecret,version,lat,long,500,500)
results = requests.get(url).json()
    




/Users/nikhil/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


**Filtering JSON file to create DataFrame**

In [25]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/Users/nikhil/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


,name,categories,lat,lng
0,Tim Hortons,Coffee Shop,43.644705,-79.567659
1,Farmer's Market Etobicoke,Farmers Market,43.643061,-79.566191
2,Loblaws,Grocery Store,43.643848,-79.560113
3,State & Main Kitchen & Bar,Restaurant,43.645778,-79.560374
4,Burnhamthorpe and The West Mall,Intersection,43.644786,-79.567065
5,West Mall Rink,Skating Rink,43.642138,-79.566218
6,Joe Fresh,Clothing Store,43.643911,-79.560126
7,Rabba,Convenience Store,43.647096,-79.563026
8,Four Seasons Place,Hotel,43.647128,-79.563009
9,Delimark Cafe,Café,43.647116,-79.562990


**Creating Clusters and Preprocessing Data for fitting**

In [20]:
nearby_venues['categories'].value_counts()
nearby_venues.drop(columns={'name'},inplace=True)
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
nearby_venues['categories'] = label.fit_transform(nearby_venues['categories'])
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
nearby_venues = scaler.fit_transform(nearby_venues)

**Fitting Dataset to clusters**

In [21]:
from sklearn.cluster import KMeans
kanalysis = KMeans(n_clusters = 5)
kanalysis.fit(nearby_venues)
labels = kanalysis.labels_

In [22]:
print(labels)

[1 1 0 2 1 4 0 3 2 3]
